### Importar Librerías 

In [1]:
%reset -f
import cudf as cf
import rmm
#import tensorflow as tf
import pandas as pd
import numpy as np
import os
from functools import partial, reduce
import operator

In [2]:
def getAllPaths(data_directory):
    #Input: directorio de los archivos.
    #Return: Arreglo con las rutas de los archivos.
    file_name = [os.path.join(data_directory,f) 
                 for f in os.listdir(data_directory)
                 if f.endswith(".csv")]
    return file_name

### Cargo todos los dataset de pruebas.

In [3]:
PATHS_CSV = getAllPaths("../data_sophia2/")
PATH_CF = cf.DataFrame(PATHS_CSV)
PATH_CF.columns=["PATH_FILES"]
PATH_CF.tail(3)

,PATH_FILES
0,../data_sophia2/chile_2021-03.csv
1,../data_sophia2/chile_2021-04.csv
2,../data_sophia2/chile_2021-02.csv


### Reducimos los dataset a una menor dimensión

In [4]:
def cargarDF(x,df):
    return cf.read_csv(df.iloc[x,0])
def reducirDF(x,df,stop,colname="text"):
    return df[x].loc[0:stop,[colname]]
def dropNA(x,df):
    return df_sample_sophia2[x].to_pandas().dropna()

### Debido a las limitaciones de Hardware en que se esta haciendo estas pruebas, solo se consideran unos pocos textos. Una vez generalizado el Código se probará con todos los datos.

In [15]:
stop_row=5
limit_debug=0
df__sophia2 = list(map(partial(cargarDF, df=PATH_CF),range(PATH_CF.shape[0]-limit_debug)))
df_sample_sophia2 =list(map(partial(reducirDF, df=df__sophia2,stop=stop_row,colname="text"),range(len(df__sophia2))))
df_sophia2_subset_pd = list(map(partial(dropNA,df=df_sample_sophia2),range(len(df_sample_sophia2))))
df_sophia2_subset = cf.DataFrame(df_sophia2_subset_pd)

In [6]:
#display([df_sophia2_subset[i].isna().sum() for i in range(len(df_sophia2_subset))])

In [7]:
#del df__sophia2, PATH_CF,df_sample_sophia2, stop_row

Se ha reducido el array de datasets para efectos prácticos, también se han eliminado los valores nulos del subset y guardaremos en nuevos archivos para que trabajemos sobre ellos posteriormente.

In [8]:
#x=[df_sophia2_subset[k].to_csv(f"../data_subset/df_sample_sophia2{k}.csv") for k in range(len(df_sophia2_subset))]

In [9]:
#Nuestro subset contiene solo los textos 
#df_sophia2_subset[0].head(5)

### Generamos los Matches con Spacy.
De acuerdo a la cantidad de cores que tengamos, lanzamos las tareas asociadas a los subset en paralelo, el cálculo de los matches de cada elemento texto de nuestro corpus

In [10]:
from joblib import Parallel, delayed
import multiprocessing

ITERATOR_DF = list(range(len(df_sophia2_subset))) # tamaño del array de datasets.
print(len(ITERATOR_DF))

def getText(x,index,df):
    #x : iterador de los textos del dataset en posicion index
    #index: indice del dataset dentro del array de dataset df_sophia2_subset
    # df : Dataframe df_sophia2_subset
    #return : span for all matcher 
    from collections import Counter
    import operator
    from functools import partial
    import spacy
    from spacy.matcher import Matcher
    #spacy.prefer_gpu()
    from functools import partial
    nlp = spacy.load("es_core_news_sm")
    #nlp = spacy.load("es_dep_news_trf")
    def counterReduce(matches, doc):
        start = matches[1]
        end = matches[2]
        span = doc[start:end]
        return span.text

    
    matcher = Matcher(nlp.vocab)
    pattern_1 = [{"POS": "NOUN"},{"LOWER": "de"}, {"POS": "NOUN"}]
    matcher.add("NOUN-de-NOUN", [pattern_1])
    pattern_2 = [{"POS": "NOUN"}, {"POS": "ADJ"}]
    matcher.add("NOUN-ADJ", [pattern_2])
    
    
    text_atomic = df[index].iloc[x,0] 
    doc_x = nlp(text_atomic)
    matches = matcher(doc_x)
    span_reduce = list(map(partial(counterReduce, doc=doc_x),matches))
    return Counter(span_reduce)


def launchTask(index_df,df):
    #index_df: iterador del array de dataset
    #df: Dataframe df_sophia2_subset
    #return: lista de arrays que contiene las construcciones terminologícas
    import ipyparallel as ipp
    from functools import partial
    rc = ipp.Client()
    dv = rc[:]
    Concepts = reduce(operator.add, list(dv.map(partial(getText,index=index_df,df=df),range(len(df[index_df])) )))
    print("Terminado")
    return Concepts

3


### Obtenemos una collections de los conceptos.

In [11]:
%%time
#n_jobs = multiprocessing.cpu_count()
n_jobs = 1
Concept_freq = Parallel(n_jobs=n_jobs)(delayed(launchTask)(i,df_sophia2_subset) for i in ITERATOR_DF)


Terminado
Terminado
Terminado
CPU times: user 104 ms, sys: 13.3 ms, total: 117 ms
Wall time: 6.94 s


### Reducimos el contador de conceptos.

In [12]:
concept_freq_total = reduce(operator.add,Concept_freq)

In [14]:
concept_freq_total

Counter({'fortunas personales': 1,
         'riqueza propuesta': 1,
         'senadora demócrata': 1,
         'legisladores progresistas': 1,
         'patrimonio neto': 1,
         'impuesto anual': 1,
         'impuesto adicional': 1,
         'cargo fiscal': 1,
         'índice de multimillonarios': 1,
         'estrictos de presentación': 1,
         'presentación de informes': 1,
         'impuesto de salida': 1,
         'jueces conservadores': 1,
         'impuesto propuesto': 1,
         'año pasado': 3,
         'arranque de año': 1,
         'inicio de año': 2,
         'economía nacional': 2,
         'partir de 1986': 1,
         'dato de enero': 1,
         'indicador mensual': 1,
         'crisis económicas': 1,
         'crisis asiática': 1,
         'mes de 2009': 1,
         'crisis gatillada': 1,
         'reapertura gradual': 1,
         'actividad económica': 4,
         'mercado laboral': 3,
         'inversiones públicas': 1,
         'reactivación económica': 1,